In [12]:
import torch
import numpy as np
from scipy.optimize import linear_sum_assignment
from numpy import *
import cv2
import scipy.io as scio
from scipy.optimize import linear_sum_assignment
import time
# from line_profiler import LineProfiler
import pickle
import networkx

In [10]:
def graphmatch_fram(adjacency1, adjacency2, K=0, erro=0.1, a=0.5, lamb=1,k=100,max_dsp_iter=1000,dsp_tol=1.1,theta=10):
    print("################### FRAM  ############")
    start = time.perf_counter()

    n, _ = adjacency1.shape
    m, _ = adjacency2.shape

    big_nm = max(n, m)
    X = np.ones((n, m)) / (n * m)
    Y = np.zeros((big_nm, big_nm))

    for i in range(30):
        x = X
        Y[0:n, 0:m] = adjacency1 @ X @ adjacency2 + lamb * K
        Y = fra(Y, theta=theta, max_ip_iter=1,max_dsp_iter=max_dsp_iter, dsp_tol=dsp_tol)
        X = (1 - a) * X + a * Y[0:n, 0:m]
        err= np.linalg.norm(x-X, ord="fro")/np.linalg.norm(X, ord="fro")
        print(i, err)
        if err < erro:
            print("FRAM converge")
            break

    P = hugarian(X)
    end = time.perf_counter()
    runtime = end - start
    return P, runtime  

def fra(Y,theta=2, max_ip_iter=20,max_dsp_iter=10,dsp_tol=1.1):
    Y = np.array(Y)
    Y = Y/Y.max()
    n=Y.shape[0]
    Y = theta/2*Y
    for j in range(max_dsp_iter):
        Y, Y_sum  = dsp_once(Y)
        if Y_sum/n< dsp_tol:
            print("FRA Converge in interation  " +  str(j))
            break
    Y = np.mat(Y)
    return Y


def dsp_once(Y):
    n=Y.shape[0]
    
    Y_row = np.sum(Y,1,keepdims=True)
    Y_col = np.sum(Y,0)
    Y_sum = Y_row.sum()

    Y=Y + (Y_sum/n + 1 - Y_row - Y_col)/n
    Y[Y < 0] = 0
    return Y, Y_sum

def hugarian(matrix):
    n,m=matrix.shape
    P=np.mat(zeros((n,m)))
    row_ind,col_ind=linear_sum_assignment(-matrix)
    P[row_ind,col_ind]=1
    return P

In [8]:
G_face = pickle.load(open("G_face.gpickle", 'rb'))
G_face_noise_5 = pickle.load(open("G_face_noise_5.gpickle", 'rb'))
G_face_noise_15 = pickle.load(open("G_face_noise_15.gpickle", 'rb'))
G_face_noise_25 = pickle.load(open("G_face_noise_25.gpickle", 'rb'))


adj = networkx.to_numpy_array(G_face)
adj_5 = networkx.to_numpy_array(G_face_noise_5)
adj_15 = networkx.to_numpy_array(G_face_noise_15)
adj_25 = networkx.to_numpy_array(G_face_noise_25)

In [13]:
P, runtime = graphmatch_fram(adj_5, adj,a=0.95, max_dsp_iter=1000,erro=0.1,theta=10)
print("Run time: "+str(runtime))
print("Node acc: "+str(sum(np.diag(P))/4039)) 
print("Edge acc: "+str(np.sum(np.multiply(adj_25,P*adj*P.T))/adj.sum()))

################### FRAM  ############
FRA Converge in interation  26
0 0.9999963285671836
FRA Converge in interation  79
1 0.709359143034408
FRA Converge in interation  109
2 0.7157163682430894
FRA Converge in interation  140
3 0.32594296881369955
FRA Converge in interation  158
4 0.196130222580062
FRA Converge in interation  167
5 0.11859490827675569
FRA Converge in interation  172
6 0.08631586747280635
FRAM converge
Run time: 224.51126709999517
Node acc: 0.9465214161921268
Edge acc: 0.9985493120565768
